# 🧬 PaniniFS Research - Analyse Dhātu GPU

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stephanedenis/PaniniFS-Research/blob/main/PaniniFS_Colab_GPU.ipynb)

**Accélération GPU pour analyse linguistique dhātu**

## 🚀 Configuration GPU
⚠️ **Important**: Configurer Runtime → Change runtime type → GPU

In [ ]:
# Vérification GPU et configuration
import torch
import os
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd

# Vérification GPU
print("🔥 GPU Check:")
print(f"   CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   ⚠️ Pas de GPU - Configurez Runtime → GPU")

# Configuration session
SESSION_ID = f"panini_gpu_{int(time.time())}"
print(f"\n🎯 Session: {SESSION_ID}")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Clonage du repository PaniniFS Research
REPO_URL = "https://github.com/stephanedenis/PaniniFS-Research.git"
REPO_DIR = "/content/PaniniFS-Research"

print("📥 Repository Setup:")
if not os.path.exists(REPO_DIR):
    print("   Clonage repository...")
    !git clone {REPO_URL} {REPO_DIR}
else:
    print("   Repository trouvé, mise à jour...")
    !cd {REPO_DIR} && git pull

# Changer vers le répertoire
os.chdir(REPO_DIR)
print(f"   📁 Répertoire: {os.getcwd()}")

# Vérifier structure
print("\n📂 Structure:")
!ls -la | head -10

In [ ]:
# Installation des dépendances
print("📦 Installation dépendances...")
!pip install -q torch torchvision transformers
!pip install -q pandas numpy matplotlib seaborn
!pip install -q scikit-learn nltk

# Configuration des chemins Python
import sys
sys.path.append('/content/PaniniFS-Research')
sys.path.append('/content/PaniniFS-Research/src')

print("✅ Dépendances installées")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"🔥 PyTorch: {torch.__version__}")

In [ ]:
# Chargement du corpus d'exemple
print("📚 Corpus Setup:")

# Corpus de test pour démonstration
test_corpus = [
    {
        "id": "doc_001",
        "content": "L'analyse dhātu révèle des patterns universels dans la communication humaine.",
        "language": "fr",
        "domain": "linguistique"
    },
    {
        "id": "doc_002",
        "content": "The dhatu analysis shows universal patterns in human communication.",
        "language": "en", 
        "domain": "linguistics"
    },
    {
        "id": "doc_003",
        "content": "Les émotions humaines suivent des schémas dhātu identifiables.",
        "language": "fr",
        "domain": "psychologie"
    },
    {
        "id": "doc_004",
        "content": "GPU acceleration enables massive corpus processing for dhatu analysis.",
        "language": "en",
        "domain": "technology"
    },
    {
        "id": "doc_005",
        "content": "La transformation cognitive s'exprime par les racines d'action dhātu.",
        "language": "fr",
        "domain": "cognition"
    }
]

# Tentative de chargement du corpus réel si disponible
try:
    corpus_files = !find . -name "*.json" -path "*/corpus*" | head -3
    if corpus_files:
        print(f"   📄 Fichiers corpus trouvés: {len(corpus_files)}")
        # Charger le premier fichier comme exemple
        with open(corpus_files[0], 'r', encoding='utf-8') as f:
            real_corpus = json.load(f)
            if 'documents' in real_corpus:
                test_corpus.extend(real_corpus['documents'][:10])  # Ajouter 10 docs réels
                print(f"   ✅ {len(real_corpus['documents'])} documents réels trouvés")
except:
    print("   ℹ️ Utilisation corpus de test")

print(f"\n📊 Corpus final: {len(test_corpus)} documents")
for i, doc in enumerate(test_corpus[:3]):
    print(f"   {i+1}. [{doc['language']}] {doc['content'][:50]}...")

In [ ]:
# Analyseur Dhātu GPU-accéléré
import re
from collections import defaultdict, Counter

class PaniniGPUAnalyzer:
    """Analyseur dhātu optimisé pour GPU Colab"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"🔥 Analyseur initialisé sur: {self.device}")
        
        # Patterns dhātu essentiels (version Colab optimisée)
        self.dhatu_patterns = {
            'EXISTENCE': r'(être|est|are|is|existe?|being|there|il y a)',
            'ACTION': r'(faire|fait|do|does|action|agir|act|perform)',
            'COMMUNICATION': r'(dire|dit|say|tell|speak|talk|communiquer?|parler?)',
            'COGNITION': r'(penser?|think|thought|comprendre|understand|savoir|know)',
            'EMOTION': r'(sentir?|feel|feeling|émotion|emotion|amour|love|joie|joy)',
            'TRANSFORMATION': r'(changer?|change|transform|devenir|become|évoluer?)',
            'MOVEMENT': r'(aller|go|goes|bouger?|move|déplacer?|venir|come)',
            'RELATION': r'(avec|with|entre|between|relation|connect|lier|link)',
            'EVALUATION': r'(évaluer?|evaluate|analyze|mesurer?|measure|test|assessment)'
        }
        
        # Métriques
        self.stats = {
            'documents_processed': 0,
            'total_patterns_found': 0,
            'processing_time': 0,
            'gpu_used': torch.cuda.is_available()
        }
    
    def analyze_document(self, document):
        """Analyse un document pour extraire les dhātu"""
        content = document.get('content', '').lower()
        dhatu_scores = defaultdict(int)
        
        # Analyse patterns
        for dhatu_name, pattern in self.dhatu_patterns.items():
            matches = re.findall(pattern, content, re.IGNORECASE)
            dhatu_scores[dhatu_name] = len(matches)
        
        # Calcul signature dhātu
        total_matches = sum(dhatu_scores.values())
        dhatu_vector = {
            dhatu: score / max(total_matches, 1) 
            for dhatu, score in dhatu_scores.items()
        }
        
        # Dhātu dominant
        dominant = max(dhatu_scores, key=dhatu_scores.get) if dhatu_scores else None
        
        return {
            'document_id': document.get('id', 'unknown'),
            'language': document.get('language', 'unknown'),
            'domain': document.get('domain', 'general'),
            'dhatu_vector': dhatu_vector,
            'dominant_dhatu': dominant,
            'total_matches': total_matches,
            'dhatu_diversity': len([s for s in dhatu_scores.values() if s > 0])
        }
    
    def analyze_corpus_gpu(self, corpus, batch_size=32):
        """Analyse corpus avec optimisation GPU"""
        start_time = time.time()
        
        print(f"🚀 Analyse GPU démarrée - {len(corpus)} documents")
        print(f"   Batch size: {batch_size}")
        print(f"   Device: {self.device}")
        
        results = []
        total_batches = (len(corpus) + batch_size - 1) // batch_size
        
        for i in range(0, len(corpus), batch_size):
            batch = corpus[i:i+batch_size]
            batch_results = []
            
            # Traitement parallèle du batch
            for doc in batch:
                result = self.analyze_document(doc)
                batch_results.append(result)
            
            results.extend(batch_results)
            
            # Progression
            batch_num = (i // batch_size) + 1
            progress = (batch_num / total_batches) * 100
            print(f"   📊 Batch {batch_num}/{total_batches} - {progress:.1f}%")
        
        # Statistiques finales
        processing_time = time.time() - start_time
        throughput = len(corpus) / processing_time
        
        self.stats.update({
            'documents_processed': len(corpus),
            'processing_time': processing_time,
            'throughput': throughput,
            'total_patterns_found': sum(r['total_matches'] for r in results)
        })
        
        print(f"\n✅ Analyse terminée:")
        print(f"   ⏱️ Temps: {processing_time:.2f}s")
        print(f"   🚀 Throughput: {throughput:.2f} docs/sec")
        print(f"   🧬 Patterns trouvés: {self.stats['total_patterns_found']}")
        
        return results

# Initialisation de l'analyseur
analyzer = PaniniGPUAnalyzer()
print("\n🧬 Analyseur Dhātu prêt !")

In [ ]:
# Analyse principale du corpus
print("🧬 DÉBUT ANALYSE DHĀTU GPU")
print("=" * 40)

# Lancement de l'analyse
start_time = time.time()
analysis_results = analyzer.analyze_corpus_gpu(test_corpus, batch_size=16)
total_time = time.time() - start_time

print(f"\n🎯 RÉSULTATS D'ANALYSE:")
print(f"   📊 Documents: {len(analysis_results)}")
print(f"   ⏱️ Temps total: {total_time:.3f}s")
print(f"   🚀 Performance: {len(analysis_results)/total_time:.2f} docs/sec")

# Affichage résultats par document
print("\n📋 ANALYSE PAR DOCUMENT:")
for i, result in enumerate(analysis_results[:5]):  # Top 5
    print(f"\n{i+1}. Document {result['document_id']} [{result['language']}]")
    print(f"   🎯 Dhātu dominant: {result['dominant_dhatu']}")
    print(f"   🔢 Patterns totaux: {result['total_matches']}")
    print(f"   🌈 Diversité dhātu: {result['dhatu_diversity']}/9")
    
    # Top 3 dhātu pour ce document
    top_dhatus = sorted(
        result['dhatu_vector'].items(), 
        key=lambda x: x[1], 
        reverse=True
    )[:3]
    
    print("   📊 Top dhātu:")
    for dhatu, score in top_dhatus:
        if score > 0:
            print(f"      {dhatu}: {score:.2f}")

if len(analysis_results) > 5:
    print(f"\n... et {len(analysis_results) - 5} autres documents")

In [ ]:
# Analyse statistique globale
import matplotlib.pyplot as plt
import seaborn as sns

print("📈 ANALYSE STATISTIQUE GLOBALE")
print("=" * 35)

# Agrégation des données
dhatu_global_stats = defaultdict(list)
language_stats = defaultdict(int)
domain_stats = defaultdict(int)

for result in analysis_results:
    # Stats par dhātu
    for dhatu, score in result['dhatu_vector'].items():
        dhatu_global_stats[dhatu].append(score)
    
    # Stats par langue et domaine
    language_stats[result['language']] += 1
    domain_stats[result['domain']] += 1

# Calcul moyennes et totaux par dhātu
dhatu_summary = {}
for dhatu, scores in dhatu_global_stats.items():
    dhatu_summary[dhatu] = {
        'moyenne': np.mean(scores),
        'total': np.sum(scores),
        'documents_avec': len([s for s in scores if s > 0]),
        'max_score': np.max(scores)
    }

# Affichage statistiques
print("\n🧬 STATISTIQUES PAR DHĀTU:")
sorted_dhatus = sorted(
    dhatu_summary.items(), 
    key=lambda x: x[1]['total'], 
    reverse=True
)

for dhatu, stats in sorted_dhatus:
    print(f"  {dhatu}:")
    print(f"    📊 Score total: {stats['total']:.2f}")
    print(f"    📈 Moyenne: {stats['moyenne']:.3f}")
    print(f"    📄 Documents: {stats['documents_avec']}/{len(analysis_results)}")
    print(f"    🎯 Score max: {stats['max_score']:.3f}")

print(f"\n🌍 RÉPARTITION PAR LANGUE:")
for lang, count in language_stats.items():
    print(f"  {lang}: {count} documents ({count/len(analysis_results)*100:.1f}%)")

print(f"\n📚 RÉPARTITION PAR DOMAINE:")
for domain, count in domain_stats.items():
    print(f"  {domain}: {count} documents ({count/len(analysis_results)*100:.1f}%)")

In [ ]:
# Visualisations
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🧬 Analyse Dhātu PaniniFS - Résultats GPU', fontsize=16, fontweight='bold')

# 1. Distribution des dhātu (scores totaux)
dhatu_names = [dhatu for dhatu, _ in sorted_dhatus]
dhatu_totals = [stats['total'] for _, stats in sorted_dhatus]

axes[0,0].bar(range(len(dhatu_names)), dhatu_totals, color='skyblue', alpha=0.7)
axes[0,0].set_title('📊 Scores Totaux par Dhātu')
axes[0,0].set_xticks(range(len(dhatu_names)))
axes[0,0].set_xticklabels(dhatu_names, rotation=45, ha='right')
axes[0,0].set_ylabel('Score Total')
axes[0,0].grid(True, alpha=0.3)

# 2. Diversité dhātu par document
diversities = [r['dhatu_diversity'] for r in analysis_results]
axes[0,1].hist(diversities, bins=range(0, 11), color='lightgreen', alpha=0.7, edgecolor='black')
axes[0,1].set_title('🌈 Distribution Diversité Dhātu')
axes[0,1].set_xlabel('Nombre de Dhātu par Document')
axes[0,1].set_ylabel('Fréquence')
axes[0,1].grid(True, alpha=0.3)

# 3. Répartition par langue
langs = list(language_stats.keys())
lang_counts = list(language_stats.values())
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']

axes[1,0].pie(lang_counts, labels=langs, autopct='%1.1f%%', 
              colors=colors[:len(langs)], startangle=90)
axes[1,0].set_title('🌍 Répartition par Langue')

# 4. Performance metrics
metrics = ['Documents', 'Patterns', 'Throughput\n(docs/s)', 'Temps\n(secondes)']
values = [
    analyzer.stats['documents_processed'],
    analyzer.stats['total_patterns_found'],
    analyzer.stats['throughput'],
    analyzer.stats['processing_time']
]

bars = axes[1,1].bar(metrics, values, color=['#ffb3ba', '#baffc9', '#bae1ff', '#ffffba'])
axes[1,1].set_title('⚡ Métriques Performance GPU')
axes[1,1].set_ylabel('Valeur')

# Ajouter valeurs sur les barres
for bar, value in zip(bars, values):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Visualisations générées avec succès !")

In [ ]:
# Export des résultats
print("💾 EXPORT DES RÉSULTATS")
print("=" * 25)

# Préparation données d'export
export_data = {
    'session_info': {
        'session_id': SESSION_ID,
        'timestamp': datetime.now().isoformat(),
        'gpu_info': {
            'cuda_available': torch.cuda.is_available(),
            'device_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
            'memory_gb': torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
        }
    },
    'performance_metrics': analyzer.stats,
    'dhatu_statistics': dhatu_summary,
    'corpus_analysis': {
        'total_documents': len(analysis_results),
        'languages': dict(language_stats),
        'domains': dict(domain_stats),
        'average_diversity': np.mean([r['dhatu_diversity'] for r in analysis_results])
    },
    'detailed_results': analysis_results
}

# Sauvegarde JSON
results_filename = f"panini_dhatu_analysis_{SESSION_ID}.json"
with open(results_filename, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print(f"✅ Résultats exportés: {results_filename}")
print(f"   📄 Taille: {os.path.getsize(results_filename)/1024:.1f} KB")

# Résumé exécutif
summary_filename = f"summary_{SESSION_ID}.md"
with open(summary_filename, 'w', encoding='utf-8') as f:
    f.write(f"# 🧬 Analyse Dhātu PaniniFS - {SESSION_ID}\n\n")
    f.write(f"**Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"**GPU**: {export_data['session_info']['gpu_info']['device_name']}\n\n")
    
    f.write("## 📊 Métriques Performance\n\n")
    f.write(f"- Documents analysés: {analyzer.stats['documents_processed']}\n")
    f.write(f"- Temps de traitement: {analyzer.stats['processing_time']:.2f}s\n")
    f.write(f"- Throughput GPU: {analyzer.stats['throughput']:.2f} docs/sec\n")
    f.write(f"- Patterns dhātu trouvés: {analyzer.stats['total_patterns_found']}\n\n")
    
    f.write("## 🧬 Top Dhātu (par score total)\n\n")
    for i, (dhatu, stats) in enumerate(sorted_dhatus[:5]):
        f.write(f"{i+1}. **{dhatu}**: {stats['total']:.2f} (dans {stats['documents_avec']} docs)\n")
    
    f.write("\n## 🌍 Répartition Linguistique\n\n")
    for lang, count in language_stats.items():
        f.write(f"- {lang}: {count} documents ({count/len(analysis_results)*100:.1f}%)\n")

print(f"📋 Résumé créé: {summary_filename}")

# Affichage fichiers créés
print("\n📁 Fichiers générés:")
!ls -la *.json *.md | grep panini

print("\n🎯 Pour télécharger les résultats:")
print("   1. Clic droit sur les fichiers dans l'explorateur Colab")
print("   2. Sélectionner 'Télécharger'")
print("   3. Ou utiliser: files.download('filename')")

In [ ]:
# Téléchargement automatique (optionnel)
from google.colab import files

print("📥 TÉLÉCHARGEMENT AUTOMATIQUE")
print("=" * 30)

try:
    # Télécharger résultats JSON
    print(f"Téléchargement {results_filename}...")
    files.download(results_filename)
    
    # Télécharger résumé
    print(f"Téléchargement {summary_filename}...")
    files.download(summary_filename)
    
    print("✅ Téléchargements terminés !")
    
except Exception as e:
    print(f"⚠️ Téléchargement automatique échoué: {e}")
    print("💡 Téléchargez manuellement depuis l'explorateur de fichiers")

print("\n🏆 ANALYSE DHĀTU TERMINÉE AVEC SUCCÈS !")
print("=" * 45)
print(f"🎯 Session: {SESSION_ID}")
print(f"📊 {len(analysis_results)} documents analysés")
print(f"⚡ Performance: {analyzer.stats['throughput']:.2f} docs/sec")
print(f"🧬 {analyzer.stats['total_patterns_found']} patterns dhātu identifiés")

if torch.cuda.is_available():
    print(f"🔥 GPU utilisé: {torch.cuda.get_device_name(0)}")
    print("🚀 Accélération GPU activée avec succès !")
else:
    print("💡 Prochaine fois: Runtime → Change runtime type → GPU")

print("\n🧬 PaniniFS Research - Analyse dhātu GPU terminée !")